# Build an Agent

- https://python.langchain.com/v0.2/docs/tutorials/agents/

# End-to-end agent

In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearch
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
  print(chunk)
  print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
  print(chunk)
  print("----")

# Setup

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

In [2]:
import getpass
import os

os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Build_an_Agent"

··········


# Tavily

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


# Define tools

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722926880, 'localtime': '2024-08-05 23:48'}, 'current': {'last_updated_epoch': 1722926700, 'last_updated': '2024-08-05 23:45', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 7.6, 'wind_kph': 12.2, 'wind_degree': 245, 'wind_dir': 'WSW', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 91, 'cloud': 41, 'feelslike_c': 12.4, 'feelslike_f': 54.3, 'windchill_c': 12.4, 'windchill_f': 54.3, 'heatindex_c': 13.3, 'heatindex_f': 55.9, 'dewpoint_c': 11.7, 'dewpoint_f': 53.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 11.7, 'gust_kph': 18.9}}"}, {'url': 'https://www.wunderground.com/hourl

# Using Language Models

In [5]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 15.1 MB/s eta 0:00:00


In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

··········


In [7]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [8]:
# 모델이 도구를 호출하도록 설정
model_with_tools = model.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [10]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_p1UVB0z8eheDizXkiMQyLvf1', 'type': 'tool_call'}]


# Create the agent

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the agent

In [13]:
# 도구(검색 엔진)를 호출하지 않는 경우
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
response["messages"]

[HumanMessage(content='hi!', id='dbee621e-d1ec-4239-a1c3-af6c17892efe'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1e1c73e6-7a54-4f9d-a39d-39400b49e22b-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [14]:
# 도구(검색 엔진)을 호출하는 경우
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='9f1499b3-959e-43b1-868a-fe1742fa76b5'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_etvWFkJzGZ4YZYd6jKnTHWGw', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6957ecfc-a42f-49e9-973e-2b4d6b9f95d3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_etvWFkJzGZ4YZYd6jKnTHWGw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': 

# Streaming Messages

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content="Sorry, as a text-based AI model I don't have real-time capabilities to provide current weather updates. I recommend checking a reliable weather forecasting website or app for the most accurate and up-to-date information.", response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 88, 'total_tokens': 130}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-789bb333-3407-421a-970d-c16a8107a6d7-0', usage_metadata={'input_tokens': 88, 'output_tokens': 42, 'total_tokens': 130})]}}
----


In [17]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
  kind = event["event"]

  if kind == "on_chain_start":
    if (event["name"] == "Agent"):
      print(f"Starting agent: {event['name']} with input: {event['data'].get('input')}")
  elif kind == "on_chain_end":
    if (event["name"] == "Agent"):
      print()
      print("--")
      print(f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}")

  if kind == "on_chat_model_stream":
    content = event["data"]["chunk"].content  # 스트림 출력
    if content:
      # Empty content in the context of OpenAI means
      # that the model is asking for a tool to be invoked
      # So we only print non-empty content
      print(content, end="|")
  elif kind == "on_tool_start":   # 에이전트 도구 호출 시작
    print("--")
    print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
  elif kind == "on_tool_end":     # 에이전트 도구 호출 종료
    print(f"Done tool: {event['name']}")
    print(f"Tool output was: {event['data'].get('output')}")
    print("--")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1722928443, \'localtime\': \'2024-08-06 00:14\'}, \'current\': {\'last_updated_epoch\': 1722927600, \'last_updated\': \'2024-08-06 00:00\', \'temp_c\': 13.0, \'temp_f\': 55.5, \'is_day\': 0, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 6.5, \'wind_kph\': 10.4, \'wind_degree\': 242, \'wind_dir\': \'WSW\', \'pressure_mb\': 1015.0, \'pressure_in\': 29.96, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 91, \'cloud\': 41, \'feelslike_c\': 12.2, \'feelslike_f\': 54.0, \'windchill_

# Adding in memory

In [18]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [19]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-89f7c470-17fc-4b6d-b245-ccbd33ddd086-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. How can I assist you further?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 108, 'total_tokens': 121}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4ac70a24-d55f-4e0c-a98d-17672e523dce-0', usage_metadata={'input_tokens': 108, 'output_tokens': 13, 'total_tokens': 121})]}}
----


In [22]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. So, I'm unable to assist with your request.", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 86, 'total_tokens': 136}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec3bd2ab-dc90-4214-9c8b-b82d6487eeb7-0', usage_metadata={'input_tokens': 86, 'output_tokens': 50, 'total_tokens': 136})]}}
----
